In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 21.1 MB/s eta 0:00:00


In [2]:
import optuna
import os

WORKSPACE_DIR = "drive/MyDrive/MIXTURE"

hyperparameter_trial_name = "finetune_second"
DB_PATH = os.path.join(WORKSPACE_DIR, "optuna.db")
storage_name = f"sqlite:///{DB_PATH}"

print("Study DB", DB_PATH)

Study DB drive/MyDrive/MIXTURE/optuna.db


In [4]:
study = optuna.create_study(
        study_name=hyperparameter_trial_name,
        direction="maximize",
        storage=storage_name,
        load_if_exists=True,
    )

best_trials = study.best_trials
len(best_trials)

[I 2025-08-07 20:06:34,714] Using an existing study with name 'finetune_second' instead of creating a new one.


1

In [5]:
best_trials[0].value

0.5138469338417053

In [6]:
import numpy as np

test_form = np.array(best_trials[0].user_attrs["test_form"])
test_form.shape

(130, 51)

In [7]:
import pandas as pd

DATA_DIR = os.path.join(WORKSPACE_DIR,"Data")

def read_and_parse(fname):
  df = pd.read_csv(os.path.join(DATA_DIR,fname)).drop(columns="components_SMILES")
  return df

test_form_df = read_and_parse("test_form.csv")
print("Test Form")
display(test_form_df.head(3))

Test Form


,stimulus,Green,Cucumber,Herbal,Mint,Woody,Pine,Floral,Powdery,Fruity,...,Phenolic,Animal,Medicinal,Cooling,Sharp,Chlorine,Alcoholic,Plastic,Ozone,Metallic
0,AA322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AA374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AA444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
desc_cols = [c for c in test_form_df.columns if not c in {"stimulus", "components_SMILES"}]
test_form_df[desc_cols] = test_form

In [10]:
OUT_DIR = os.path.join(WORKSPACE_DIR,"Output","second_tuning")
os.makedirs(OUT_DIR, exist_ok=True)
test_form_df.to_csv(os.path.join(OUT_DIR,"test_form.csv"))